In [1]:
from pprint import *
from ib_insync import *
util.startLoop()  # uncomment this line when in a notebook

ib = IB() # making a IB class
# ib.connect('127.0.0.1', 7497, clientId=2)
ib.connect('10.0.0.209', 7497, clientId=3)

<IB connected to 10.0.0.209:7497 clientId=3>

Error 10089, reqId 11: Requested market data requires additional subscription for API. See link in 'Market Data Connections' dialog for more details.Delayed market data is available.NFLX NASDAQ.NMS/TOP/ALL, contract: Stock(conId=15124833, symbol='NFLX', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='NFLX', tradingClass='NMS')
Error 103, reqId 18: Duplicate order id
Error 103, reqId 19: Duplicate order id
Error 103, reqId 19: Duplicate order id
Error 104, reqId 23: Cannot modify a filled order.
Error 104, reqId 31: Cannot modify a filled order.
Value (Trade(contract=Stock(conId=15124833, symbol='NFLX', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='NFLX', tradingClass='NMS'), order=MarketOrder(orderId=40, clientId=3, permId=2124326699, action='BUY', totalQuantity=2.0, lmtPrice=0.0, auxPrice=0.0), orderStatus=OrderStatus(orderId=40, status='Filled', filled=2.0, remaining=0.0, avgFillPrice=636.95, permId=2124326699, parentId=0, lastFillP

In [2]:
nflx_contract = Stock('NFLX', 'SMART', 'USD')
ib.qualifyContracts(nflx_contract)

[Stock(conId=15124833, symbol='NFLX', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='NFLX', tradingClass='NMS')]

In [3]:
data = ib.reqMktData(nflx_contract)

In [4]:
data.marketPrice()

nan

In [5]:
eur_usd_contract = Forex('EURUSD', 'IDEALPRO')
ib.qualifyContracts(eur_usd_contract)

[Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD')]

In [6]:
btc_fut_contract = Future('BRR', '20240426', 'CME')
ib.qualifyContracts(btc_fut_contract)


[Future(conId=662532985, symbol='BRR', lastTradeDateOrContractMonth='20240426', multiplier='5', exchange='CME', currency='USD', localSymbol='BTCJ4', tradingClass='BTC')]

In [8]:
btc_fut_cont_contract = ContFuture('BRR', 'CME')
ib.qualifyContracts(btc_fut_cont_contract)

[ContFuture(conId=662532985, symbol='BRR', lastTradeDateOrContractMonth='20240426', multiplier='5', exchange='CME', currency='USD', localSymbol='BTCJ4', tradingClass='BTC')]

In [9]:
gold_cfd_contract = CFD('XAUUSD')
ib.qualifyContracts(gold_cfd_contract)

[CFD(conId=457068913, symbol='XAUUSD', exchange='SMART', currency='USD', localSymbol='XAUUSD', tradingClass='XAUUSD')]

In [10]:
historical_data_nflx = ib.reqHistoricalData(
    nflx_contract,
    '',
    barSizeSetting='15 mins',
    durationStr='2 D',
    whatToShow='MIDPOINT',
    useRTH=True
)

In [11]:
historical_data_nflx

[BarData(date=datetime.datetime(2024, 4, 3, 9, 30, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')), open=612.0, high=618.32, low=611.78, close=616.09, volume=-1.0, average=-1.0, barCount=-1),
 BarData(date=datetime.datetime(2024, 4, 3, 9, 45, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')), open=616.09, high=616.44, low=614.41, close=615.97, volume=-1.0, average=-1.0, barCount=-1),
 BarData(date=datetime.datetime(2024, 4, 3, 10, 0, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')), open=615.97, high=621.15, low=615.97, close=620.85, volume=-1.0, average=-1.0, barCount=-1),
 BarData(date=datetime.datetime(2024, 4, 3, 10, 15, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')), open=620.85, high=623.81, low=620.82, close=623.32, volume=-1.0, average=-1.0, barCount=-1),
 BarData(date=datetime.datetime(2024, 4, 3, 10, 30, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')), open=623.32, high=623.85, low=621.81, close=623.37, volume=-1.0, average=-1.0, barCount=-1),
 BarData(date=datetime.datetime(2024, 4, 3, 10, 45, tz

In [13]:
type(historical_data_nflx)

ib_insync.objects.BarDataList

In [14]:
historical_data_nflx[-1]

BarData(date=datetime.datetime(2024, 4, 4, 11, 0, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')), open=633.77, high=634.68, low=633.76, close=634.38, volume=-1.0, average=-1.0, barCount=-1)

In [15]:
historical_data_nflx[-1].open

633.77

In [17]:
from pandas import *

In [18]:
util.df(historical_data_nflx)

,date,open,high,low,close,volume,average,barCount
0,2024-04-03 09:30:00-04:00,612.00,618.32,611.78,616.09,-1.0,-1.0,-1
1,2024-04-03 09:45:00-04:00,616.09,616.44,614.41,615.97,-1.0,-1.0,-1
2,2024-04-03 10:00:00-04:00,615.97,621.15,615.97,620.85,-1.0,-1.0,-1
3,2024-04-03 10:15:00-04:00,620.85,623.81,620.82,623.32,-1.0,-1.0,-1
4,2024-04-03 10:30:00-04:00,623.32,623.85,621.81,623.37,-1.0,-1.0,-1
5,2024-04-03 10:45:00-04:00,623.37,625.90,623.37,625.65,-1.0,-1.0,-1
6,2024-04-03 11:00:00-04:00,625.65,628.14,625.65,627.11,-1.0,-1.0,-1
7,2024-04-03 11:15:00-04:00,627.11,627.82,626.23,626.79,-1.0,-1.0,-1
8,2024-04-03 11:30:00-04:00,626.79,627.31,625.82,626.07,-1.0,-1.0,-1
9,2024-04-03 11:45:00-04:00,626.07,626.07,623.64,624.30,-1.0,-1.0,-1


In [19]:
nflx_df = util.df(historical_data_nflx)

In [20]:
nflx_df.close.rolling(20).mean()

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
5          NaN
6          NaN
7          NaN
8          NaN
9          NaN
10         NaN
11         NaN
12         NaN
13         NaN
14         NaN
15         NaN
16         NaN
17         NaN
18         NaN
19    624.8210
20    625.4060
21    625.9445
22    626.2135
23    626.4025
24    626.6450
25    626.8760
26    627.1585
27    627.5265
28    627.9005
29    628.3910
30    628.8125
31    629.1780
32    629.5900
Name: close, dtype: float64

In [129]:
nflx_df['20SMA'] = nflx_df.close.rolling(20).mean()

In [26]:
nflx_df.close.rolling(20).mean().iloc[-1]

629.59

### How can I fire an order through ib_insync?


In [179]:
nflx_order = MarketOrder('BUY',2)

In [180]:
import time
time.sleep(10)
trade = ib.placeOrder(nflx_contract, nflx_order)

In [175]:
trade.log

[TradeLogEntry(time=datetime.datetime(2024, 4, 4, 16, 48, 32, 39956, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0)]

In [177]:
for _ in range(1000):
    if not trade.isActive():
        print('Your order status - {0}'.format(trade.orderStatus.status))
        break
    time.sleep(0.5)
else:
    print('Order is still active')

Order is still active


In [182]:
def order_status(trade):
    if trade.OrderStatus.status == 'Filled':
        fill = trade.fills[-1]
        print('{0} - {1} {2} {3} @ {4}'.format(fill.time, fill.execution.side, fill.contract.symbol, fill.execution.shares, fill.execution.avgPrice))

In [92]:
trade.filledEvent

Event<filledEvent, []>

In [183]:
trade.filledEvent += order_status

In [198]:
trade.filledEvent

Event<filledEvent, [[None, None, <function order_status at 0x13459f040>]]>

eur_usd_contract = Forex('EURUSD', 'IDEALPRO')
ib.qualifyContracts(eur_usd_contract)

In [95]:
eur_usd_order = ib.bracketOrder(
    'BUY',
    1000,
    limitPrice=1.19,
    takeProfitPrice=1.20,
    stopLossPrice=1.18
)

In [96]:
for ord in eur_usd_order:
    ib.placeOrder(eur_usd_contract, ord)

In [97]:
visa_contract = Stock('V', 'SMART', 'USD')
ib.qualifyContracts(visa_contract)

[Stock(conId=49462172, symbol='V', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='V', tradingClass='V')]

In [98]:
price_condition = PriceCondition(
    price=200,
    conId=visa_contract.conId,
    exch=visa_contract.exchange
)

In [99]:
ma_contract = Stock('MA', 'SMART', 'USD')
ib.qualifyContracts(ma_contract)

[Stock(conId=38685693, symbol='MA', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='MA', tradingClass='MA')]

In [100]:
visa_ma_order = MarketOrder('BUY', 10)
visa_ma_order.conditions.append(price_condition)

In [101]:
visa_ma_trade = ib.placeOrder(ma_contract, visa_ma_order)

In [102]:
visa_ma_trade.isActive()

True